In [23]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Initialize neuron functions dynamically
def create_neuron(weights, bias, activation, is_final_layer=False):
    activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
    
    def neuron(inputs):
        z = np.dot(weights, inputs) + bias
        return z if is_final_layer else activation_func(z)
    
    neuron.weights = weights  # Store weights for debugging
    neuron.bias = bias  # Store bias for debugging
    neuron.activation = activation  # Store activation type
    return neuron

# Build network dynamically from JSON structure
def build_network(json_data):
    layers = []  # Store network layers
    
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order
    
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        layer_neurons = []
        
        for node in layer_info['nodes']:
            weights = np.array(node['weights'])
            biases = np.array(node['biases'])
            activation = node['activation']
            
            neuron_func = create_neuron(weights, biases, activation, is_final_layer=(i == len(sorted_layers) - 1))
            layer_neurons.append(neuron_func)
        
        layers.append(layer_neurons)
    
    return layers

# Forward pass with Debugging
def forward_pass(layers, input_data):
    x = np.array(input_data)
    #print("Input to network:", x[:10])  # Debugging: Check first 10 inputs
    
    # Pass through each layer
    for i, layer in enumerate(layers):
        raw_output = np.array([np.dot(neuron.weights, x) + neuron.bias for neuron in layer])  # Pre-activation
        #print(f"Layer {i} raw output:", raw_output[:10])  # Debugging: Before activation
        
        x = np.array([neuron(x) for neuron in layer])
        #print(f"Layer {i} output after activation:", x[:10])  # Debugging: After activation
        
        # Apply softmax only at final layer
        if i == len(layers) - 1:
            x = softmax(x)
            #print("Final softmax output:", x)  # Debugging final output
    
    return np.argmax(x)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(network, image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9
